In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset

c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("SKNahin/bengali-transliteration-data")
train_test_split =dataset["train"].train_test_split(test_size=0.2, seed=42)
train = train_test_split["train"]
test = train_test_split["test"]

In [3]:
def preprocess_data(batch):
    inputs = [f"transliterate: {text}" for text in batch["bn"]]
    targets = batch["rm"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

In [6]:
model_name="t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataset = train.map(preprocess_data, batched=True)
val_dataset = test.map(preprocess_data, batched=True)

In [10]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [11]:
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])